Visiblement on overfit beaucoup, on va tester des modèles avec moins de données. 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.pipeline import make_pipeline
from skrub import TableVectorizer

Lecture des données

In [2]:
PATH = "./dataproject2025.csv"
df = pd.read_csv(PATH).drop(columns=["Unnamed: 0"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1086236 entries, 0 to 1086235
Data columns (total 38 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   issue_d                  1086236 non-null  int64  
 1   loan duration            1086236 non-null  int64  
 2   annual_inc               1086236 non-null  float64
 3   avg_cur_bal              1086236 non-null  float64
 4   bc_open_to_buy           1086236 non-null  float64
 5   bc_util                  1086236 non-null  float64
 6   delinq_2yrs              1086236 non-null  float64
 7   dti                      1086236 non-null  float64
 8   emp_length               1086236 non-null  object 
 9   emp_title                1086236 non-null  object 
 10  fico_range_high          1086236 non-null  float64
 11  funded_amnt              1086236 non-null  float64
 12  grade                    1086236 non-null  object 
 13  home_ownership           1086236 non-null 

In [5]:
%load_ext autoreload
from utils import ordinal_encode
from sklearn.model_selection import cross_validate, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LinearRegression
from custom_sequential_feature_selector import MlxtendSFSWrapper

X = ordinal_encode(
        df
        .sort_values(by="issue_d", ascending=True)
        .drop(columns=["Predicted probabilities", "Predictions", "target"])
        # low information, must be redundant
        .drop(columns=["emp_title"]), 
        drop=True)

# mixed effect like
X["issue_d"] = X["issue_d"].astype(str)

y = df["Predicted probabilities"]

# Identifier les colonnes numériques et catégorielles
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

# --- Préprocesseur ---
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop"  # on ne garde que ce qui est défini
)

cv = KFold(n_splits=5, shuffle=True, random_state=42)

sfs = MlxtendSFSWrapper(
    LinearRegression(),
    forward=True,
    cv=cv,
    scoring="r2",
    n_jobs=2, 
    verbose=2
)

model = Pipeline([
    ("preprocessor", preprocessor),
    ("feature_selection", sfs),
    ("regressor", LinearRegression())
])

model.fit(X,y)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   19.3s
[Parallel(n_jobs=2)]: Done  56 out of  56 | elapsed:   29.3s finished

[2025-09-28 14:24:37] Features: 1/56 -- score: 0.003522106136205472[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   37.4s
[Parallel(n_jobs=2)]: Done  55 out of  55 | elapsed:   54.6s finished

[2025-09-28 14:25:32] Features: 2/56 -- score: 0.008219933970077541[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   40.4s
[Parallel(n_jobs=2)]: Done  54 out of  54 | elapsed:   59.9s finished

[2025-09-28 14:26:32] Features: 3/56 -- score: 0.009034756296959313[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   53.0s
[Parallel(n_jobs=2)]: Done  53 out of  53 

AttributeError: SequentialFeatureSelector has not been fitted, yet.